In [1]:
import pyspark
from delta import configure_spark_with_delta_pip

In [2]:
builder = pyspark.sql.SparkSession.builder.appName("MyApp") \
    .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension") \
    .config("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog")

In [3]:
spark = configure_spark_with_delta_pip(builder).getOrCreate()

23/01/30 00:42:59 WARN Utils: Your hostname, Noels-MacBook-Pro.local resolves to a loopback address: 127.0.0.1; using 192.168.0.14 instead (on interface en0)
23/01/30 00:42:59 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
:: loading settings :: url = jar:file:/Users/noel/PycharmProjects/data-engineering-workbench/venv/lib/python3.9/site-packages/pyspark/jars/ivy-2.5.0.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /Users/noel/.ivy2/cache
The jars for the packages stored in: /Users/noel/.ivy2/jars
io.delta#delta-core_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-cfb2f31f-75f2-4c63-9b99-8083b3b392c1;1.0
	confs: [default]
	found io.delta#delta-core_2.12;2.2.0 in central
	found io.delta#delta-storage;2.2.0 in central
	found org.antlr#antlr4-runtime;4.8 in central
downloading https://repo1.maven.org/maven2/io/delta/delta-core_2.12/2.2.0/delta-core_2.12-2.2.0.jar ...
	[SUCCESSFUL ] io.delta#delta-core_2.12;2.2.0!delta-core_2.12.jar (3023ms)
downloading https://repo1.maven.org/maven2/io/delta/delta-storage/2.2.0/delta-storage-2.2.0.jar ...
	[SUCCESSFUL ] io.delta#delta-storage;2.2.0!delta-storage.jar (729ms)
downloading https://repo1.maven.org/maven2/org/antlr/antlr4-runtime/4.8/antlr4-runtime-4.8.jar ...
	[SUCCESSFUL ] org.antlr#antlr4-runtime;4.8!antlr4-runtime.jar (1218ms)
:: resolution report :: resolve 7154ms :: artifacts dl 

23/01/30 00:43:12 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


# Create a table

To create a Delta table, write a DataFrame out in the delta format. You can use existing Spark SQL code and change the format from parquet, csv, json, and so on, to delta.

In [4]:
data = spark.range(0, 5)
data.write.format("delta").save("/tmp/delta-table", source='parquet')

23/01/30 00:43:19 WARN package: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


AnalysisException: Cannot write to already existent path file:/tmp/delta-table without setting OVERWRITE = 'true'.

These operations create a new Delta table using the schema that was inferred from your DataFrame. For the full set of options available when you create a new Delta table, see Create a table and Write to a table.

> Note: This quickstart uses local paths for Delta table locations. For configuring HDFS or cloud storage for Delta tables, see Storage configuration.


# Read data


You read data in your Delta table by specifying the path to the files: "/tmp/delta-table":

In [7]:
df = spark.read.format("delta").load("/tmp/delta-table")
df.show()

+---+
| id|
+---+
|  6|
|  5|
|  7|
|  8|
|  9|
+---+



In [11]:
df.to_pandas_on_spark()

/Users/noel/PycharmProjects/data-engineering-workbench/venv/lib/python3.9/site-packages/pyspark/sql/dataframe.py:3313: FutureWarning: DataFrame.to_pandas_on_spark is deprecated. Use DataFrame.pandas_api instead.
  warnings.warn(


,id
0,6
1,5
2,7
3,8
4,9


# Update table data

Delta Lake supports several operations to modify tables using standard DataFrame APIs. This example runs a batch job to overwrite the data in the table:

## Overwrite

In [ ]:
data = spark.range(5, 10)
data.write.format("delta").save("/tmp/delta-table", source='parquet', mode='overwrite')

In [ ]:
df = spark.read.format("delta").load("/tmp/delta-table")
df.show()